# Realtime Hand Sign translation with CNN

## Initial setup - imports, loading data, utility functions

In [ ]:
%pip install mediapipe opencv-python matplotlib tensorflow

In [7]:
# imports for data processing
import os
import mediapipe as mp
import numpy as np
import cv2

In [8]:
# data dir paths
unprocessed_data_dir = 'data/unprocessed'
processed_data_dir = 'data/processed'

# media pipe hands object
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9
)

In [12]:
# function to create image with landmarks
def get_hand_landmarks_image(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # draw landmarks on blank 224x224 white image
    output_img = np.ones((224, 224, 3), np.uint8) * 255

    try:
        results = hands.process(image)
    except ValueError:
        return False, output_img

    mp_drawing = mp.solutions.drawing_utils

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                output_img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=3, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(20,20,20), thickness=3, circle_radius=1),
            )
    return not results.multi_hand_landmarks == None, output_img

### Create data with landmarked hand images from data/unprocessed (Optional)

In [9]:
# loop through all directores in unprocessed_data_dir
for dir in os.listdir(unprocessed_data_dir):
    # create output dir
    output_dir = os.path.join(processed_data_dir, dir)
    os.makedirs(output_dir, exist_ok=True)
    
    # loop through all files in dir
    for file in os.listdir(os.path.join(unprocessed_data_dir, dir)):
        # load image
        image = cv2.imread(os.path.join(unprocessed_data_dir, dir, file))
        
        # process image
        ret, processed_image = get_hand_landmarks_image(image)

        # save image
        if ret:
            cv2.imwrite(os.path.join(output_dir, file), processed_image)
        else:
            print(f'No hands detected in {dir}/{file}')
        
    print(f'Processed {dir} directory')
        
# release hands object
hands.close()

c:\Users\Kamin\fyps\hand-sign-rt\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed 0 directory
Processed 1 directory
Processed 10 directory
Processed 11 directory
No hands detected in 12/11.jpg
No hands detected in 12/110.jpg
No hands detected in 12/126.jpg
No hands detected in 12/14.jpg
No hands detected in 12/15.jpg
No hands detected in 12/150.jpg
No hands detected in 12/151.jpg
No hands detected in 12/152.jpg
No hands detected in 12/153.jpg
No hands detected in 12/154.jpg
No hands detected in 12/155.jpg
No hands detected in 12/156.jpg
No hands detected in 12/157.jpg
No hands detected in 12/158.jpg
No hands detected in 12/159.jpg
No hands detected in 12/161.jpg
No hands detected in 12/162.jpg
No hands detected in 12/163.jpg
No hands detected in 12/164.jpg
No hands detected in 12/165.jpg
No hands detected in 12/166.jpg
No hands detected in 12/167.jpg
No hands detected in 12/168.jpg
No hands detected in 12/169.jpg
No hands detected in 12/174.jpg
No hands detected in 12/177.jpg
No hands detected in 12/178.jpg
No hands detected in 12/179.jpg
No hands detected

In [13]:
# create blank image when no hands are detected
output_img = np.ones((224, 224, 3), np.uint8) * 255
for i in range(1000):
    cv2.imwrite(os.path.join('data', 'processed', '26', f'{i}.jpg'), output_img)

## ASL CNN model

### Define and train model

In [1]:
# imports for model training/testing
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'data/processed',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    'data/processed',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 21400 images belonging to 27 classes.
Found 5348 images belonging to 27 classes.


In [3]:
# define model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(27, activation='softmax')
])

c:\Users\Kamin\fyps\hand-sign-rt\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
669/669 ━━━━━━━━━━━━━━━━━━━━ 400s 595ms/step - accuracy: 0.7384 - loss: 0.9449 - val_accuracy: 0.9678 - val_loss: 0.1593
Epoch 2/10
399/669 ━━━━━━━━━━━━━━━━━━━━ 2:32 565ms/step - accuracy: 0.9867 - loss: 0.0381

KeyboardInterrupt: 

In [25]:
model.save('model.keras')

### Test model

In [26]:
# load model
model = load_model('model.keras')

### Test static image

In [28]:
# test image
image = cv2.imread('data/processed/21/959.jpg')

added_dim_img = np.expand_dims(image, axis=0)

prediction = model.predict(added_dim_img)
predicted_class = np.argmax(prediction)

print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Predicted class: 14


### Test realtime video from webcam

In [16]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # Preprocess the frame to match the input size of the model and scale the pixel values
    frame = cv2.resize(frame, (224, 224))

    ret, hand_landmarks_img = get_hand_landmarks_image(frame)
    
    # Add an extra dimension because the model expects batches of images
    added_dim_img = np.expand_dims(hand_landmarks_img, axis=0)

    # Use the model to predict the class of the frame
    prediction = model.predict(added_dim_img)

    # Get the class with the highest probability
    predicted_class = np.argmax(prediction)

    # Display the predicted class on the frame and print it
    cv2.putText(hand_landmarks_img, chr(65 + predicted_class), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Hand Landmarks', hand_landmarks_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [15]:
cap.release()
cv2.destroyAllWindows()